In [1]:
import sys
sys.path.append('..')

In [2]:
from agents.dqn import *
from core.replay import *
from environments.wrappers import *
from networks.cnn import *
from utils.train import *

In [3]:
import gym
import torch

### Network, Environment, Agent

In [4]:
network = DQNCNN()

In [5]:
# parameters
size = (84, 84)
num_stacked_frames = 4

raw_env = gym.make('BreakoutNoFrameskip-v4')
env = AtariWrapper(raw_env, size=size, k=num_stacked_frames)

In [6]:
observation_space = raw_env.observation_space
action_space = raw_env.action_space
params = {'epsilon':1.0, 'epsilon_decay':0.99, 'epsilon_min':0.05, 'gamma':0.99, 'alpha':2.5e-5,
          'memory_size':250000, 'device':'cuda', 'network':network, 'batch_size':32, 'tnet_update':10000}

agent = DQN(observation_space, action_space, **params)

### Train

In [7]:
episodes = 100000
e_verbose = 5000
start = 10000

In [ ]:
train(agent, env, episodes, e_verbose, start)

*** episode: 0, average reward: 2.0, optim steps: 0, memory: 205 ***


C:\Users\minha\AppData\Local\Temp\ipykernel_1664\507295942.py:1: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  train(agent, env, episodes, e_verbose, start)
